<a href="https://colab.research.google.com/github/zachhom/170-P3-Neural-Net/blob/Haneul/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Writeup

For the writeup, please include the following information:


*   Describe the architecture of your neural network
*   Discuss how you tuned your network and why you think it's performance is reasonable for this task
*   Discuss whether or not you feel that this classifier is appropriate for the given task (check PA3 description)



# Writeup

### `Describe the architecture of your neural network`

Our architecture uses the sequentially-layered architecture that adds the (Convolution, Max Pooling) as a C-MP combination and Dense layer after two loops of layer propagation from C-MP layers, we further add Dense Layer to the model before the output layer.

### `Discuss how you tuned your network and why you think it's performance is reasonable for this task`

The overall idea is to `increase accuracy value` while `decresing value loss`. Based on the overall trend while epochs being run, we attempted to tune the basic primitives constants like `Epoch` and `Batch Size Train`.

From our understanding, we have the best result when we operate on epoch/batch_train_size. ratio of nearly 1/100 or 0.01. From our understanding, each epoch will train 100 units from the training dataset. We thought that this makes sence because we have 10 categories to disclose the output type. Thus we thought we would be training every category at least 5 times in an epoch loop. 

We get overall trend of value accuracy about 92 % and the value lost around at 0.17~0.20. This was the best overall set of data values among these categories.


### `Discuss whether or not you feel that this classifier is appropriate for the given task (check PA3 description)`

The performance for our network was 92%. Given a large dataset such as fashionMNIST, an 8% failure rate is a reasonable classifier because that means we have a relatively high success rate. Given that this is a clothing identification network, we can accept an error rate such as 8%. However, if the task was different such as facial recognition for a counter terrorism agency, 8% failure rate would be too high.


# Tensorflow

## 0. Imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

## 1. Set Hyperparameters

In [2]:
# Tune these

n_epochs = 125
batch_size_train = 4500
batch_size_test = 10000
learning_rate = 0.01

## 2. Import Data

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size_train)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size_test)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## 3. Define Model

In [4]:
model = tf.keras.models.Sequential()

# Convolutional Layers
# model.add(tf.keras.layers.Conv2D(20, (2,2), padding='same', activation='relu', input_shape=(28,28,1)))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
# model.add(tf.keras.layers.Conv2D(40, (2,2), padding='same', activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3)) 

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))


model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))


model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Dense Layers
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(32, activation='relu'))
# model.add(tf.keras.layers.Dense(16, activation='relu')) #256
# model.add(tf.keras.layers.Dense(126, activation='relu'))
# model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 32)          0

Things to consider:

* dropout, convensation, things like that
* overtrain




## 4. Train Model

In [5]:
model.fit(
    ds_train,
    epochs=n_epochs,
    validation_data=ds_test,
)


Epoch 1/125


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


14/14 [==============================] - 11s 259ms/step - loss: 1.5492 - sparse_categorical_accuracy: 0.4287 - val_loss: 0.7093 - val_sparse_categorical_accuracy: 0.7339
Epoch 2/125
14/14 [==============================] - 2s 132ms/step - loss: 0.7058 - sparse_categorical_accuracy: 0.7337 - val_loss: 0.5357 - val_sparse_categorical_accuracy: 0.7877
Epoch 3/125
14/14 [==============================] - 2s 131ms/step - loss: 0.5606 - sparse_categorical_accuracy: 0.7872 - val_loss: 0.4589 - val_sparse_categorical_accuracy: 0.8225
Epoch 4/125
14/14 [==============================] - 2s 140ms/step - loss: 0.4971 - sparse_categorical_accuracy: 0.8135 - val_loss: 0.4154 - val_sparse_categorical_accuracy: 0.8445
Epoch 5/125
14/14 [==============================] - 2s 132ms/step - loss: 0.4552 - sparse_categorical_accuracy: 0.8321 - val_loss: 0.3779 - val_sparse_categorical_accuracy: 0.8563
Epoch 6/125
14/14 [==============================] - 2s 131ms/step - loss: 0.4177 - sparse_categorical_acc